# Data formats

## Learning Outcomes

By the end of this notebook you will know how to:
- Open text and binary files.
- Open pickle files.
- Open and save `numpy` files.
- Open fits files.
- Open HDF5 files.

In the following notebooks we'll delve into some important concepts and use cases which you'll see throughout your degree and beyond. Before that, we'll cover some more common data formats and how to open them in Python. We've already seen CSV files and opened them with `pandas`, but there are many more data formats and CSV files are often cumbersome depending on your use case.

Regardless of what field you end up specialising in it's certain you'll be handling data in a range of different formats. There is an assortment of standard formats in addition to CSV files, including text files, `pickle` files, `numpy` files, fits files and hdf5 files. There are also a plethora of other formats used in specifics fields for specific functions; for instance the Met Office use a data format modelled after HDF5 files called netCDF4 files which they use for storing weather data.

In this section we will cover the formats which you will come across again and again. Hopefully this introduction, and the demonstrations it contains, will mean you will be unfazed when presented with a file to open for an assignment or project in the future. 

You can think of this notebook as reference material, unlike the other notebooks there will be no exercises.

## `with` and the importance of closing files

One of the most frustrating things you can encounter is when you've spent hours running code to make some output file only for it to become corrupted due to careless handling down line. This isn't a big deal for a file which is backed up or can easily be reproduced but in many instances you might be working with a one off file containing terabytes of data which took days, if not weeks or months, to produce. It is therefore very important to learn how to correctly handle data files. The most important thing to remember is if you have opened a file you must close it! Many data files will allow you to do this manually with some form of `close` function. For instance in HDF5 you can open and close a file like this:


In [ ]:
# Open a file
hdf = h5py.File('myfile.hdf5', 'r')

# Do some stuff with the contents of the file
do_stuff(hdf)

hdf.close()

Doing this safely closes the file, preventing possible corruption from closing improperly. 

The above method is implemented in a number of modules for handling different file formats but how it works is entirely format specific, and also specific to the module being used to handle that format. For instance, if you think back to `pandas` you may be confused why we didn't close anything there. In fact, `pandas` handles the opening and closing of files itself behind the scenes, a file is opened, parsed creating the dataframe, and then closed.

Thankfully Python includes the `with` statement structure. We've already seen this used with the CSV module in [the pandas notebook](8_pandas.ipynb) but it can be used with any file in the open function. Below we will use it for a number of the presented formats.

The following sections contain demonstrations of how to open different file types. The examples in these sections will all error since you do not have the files quoted within them, they are simply provided here for reference.

## Text and binary files

With simple text or binary files (these are files which aren't human readable but are saved with strings of binary or hex) we can use the Python open function combined with the with statement. The open function takes a file path as the first argument and a "mode" as the second argument. This mode can be `'r'` to read a file, `'w'` to write a file (and overwrite an existing file), `'a'` to append to an existing file (read and write) or `'x'` to create and write to a file, the last of these will error if the file already exists. To read a binary file you simply include `'b'` in the mode, i.e. `'rb'` to read a binary file.  


In [ ]:

with open('myfile.txt', 'r') as a:
    x = a.read()



## Pickle files

If we have a **small** dictionary or other Python object and we want a simple way to save it, the best option is to use a pickle file. This format "serialises" the data which you can think of as applying some compression. Pickle files can therefore be great when you have a sizeable amount of data and want to store it with as small a footprint as possible. 

The downside of pickle files is that when saving and loading your data you necessarily have to read/write all of it at once. This is far from ideal for huge datasets since you spend all your time reading or writing data. When your dataset is big enough this lost time is significant and there are much better formats than pickle files for these cases. 

The extension for a pickle file can effectively be anything (the same is true for binary files or text files) but convention is to use `.pkl` or `.pck` which we adopt here.

To use pickle files we need to import the `pickle` module and call it's `load` function while using `open` in binary mode.


In [ ]:
import pickle

with open('myfile.pck', 'rb') as a:  # open file
    x = pickle.load(a)  # store data in variable 



To save a pickle file we simply exchange `'rb'` for `'wb'`, `load` for `dump` and provide the variable for saving as the first argument to `dump` and the open file (here `a`) as the second.


In [ ]:
import pickle

with open('myfile.pck', 'wb') as a:  # create file object
    pickle.dump(x, a)  # store data in file


## `numpy` files

`numpy` also provides functions to save and load arrays as well as its own format. These functions do this in nice neat single line statements. In addition to the `numpy` specific format (`.npy`) it also has an interface for text files but do be aware that text files will be loaded containing strings rather than floats or integers unless explictly told otherwise.


In [ ]:

import numpy as np
        
# Load a numpy file
x = np.load('mynumpyfile.npy')

# Load a text file with numpy
x = np.loadtxt('mytextfile.txt', dtype=int)  # here we have told numpy the data is integers

# Save a numpy file
np.save('mynumpyfile.npy', y)

# Save a text file with numpy
np.savetxt('mytextfile.txt', y)


## FITs files

FITs file are widely used to store image data in astronomy (for better or worse). They are structured with "sections" containing various bits of data. They have a header which contains metadata about the images and can contain 1 or more images in different "sections". The `astropy` package contains a function for handling these files.


In [ ]:
from astropy.io import fits
        
# Open the fits file
with fits.open('myimg.fits') as hdu:

        # Get the first image in the file and store it in a variable
        img = hdu[0].data




A detailed look at this module and file format can be found [in the docs](https://docs.astropy.org/en/stable/io/fits/).



## HDF5 files

HDF5 files are becoming the norm for storing large amounts of data, I use them on a day to day basis. They are very time and memory efficient allowing a user to access subsets of huge datasets without waiting for the entire file to load (as would be the case for any of the above formats). They are structured exactly like dictionaries with keys representing "groups"  with "datasets" and "attributes" stored within groups. These groups can then also be nested just like a dictionary. To access a HDF5 file we can use the `h5py` module.


In [ ]:

import h5py

# Open a HDF5 file in read mode
hdf = h5py.File('myfile.hdf5', 'r')

# Open 2 groups
grp1 = hdf["myfirstgroupkey"]
grp2 = hdf["mysecondgroupkey"]

# Extract a dataset from group 1
dset = grp1["thefirstdataset"]

# This can also be done from the root
dset = hdf["myfirstgroupkey"]["thefirstdataset"]

# Get an attribute from group 2
at = grp2.attr["thefirstattribute"]

# Again this can be done from the root group
at = hdf["mysecondgroupkey"].attr["thefirstattribute"]

# Close the file
hdf.close()


In the above we read a file in read mode (`"r"`). We can use various modes, just like when using the `open` function. To write a file we can use write mode (`"w"`) and `h5py`'s functions for creating groups and datasets, and the syntax for writing attributes.

In [ ]:

# Open a HDF5 file in write mode
hdf = h5py.File("myfile.hdf5", "r")

# Make a group
grp1 = hdf.create_group("myfirstgroup")

# Write a dataset
grp1.create_dataset(data, shape=data.shape, dtype=int, compression="gzip")

# Write an attribute
grp1.attr["myattribute"] = 10

# Close the file
hdf.close()



More details on using `h5py` and the structure of HDF5 files can be found in their [documentation](http://docs.h5py.org/en/stable/quick.html).